# LLM Calls

In [18]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [19]:
llm = OpenAI()
chat_model = ChatOpenAI()
llm.predict("hi!")

'\\nHi!\\nGood Morning!\n\nHello! Good Morning!'

In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Download competitions

In [11]:
api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
                          file_name='Competitions.csv',
                          path='./')

True

In [12]:
import zipfile

zip_path = './Competitions.csv.zip'  # Chemin vers le fichier .zip
extract_path = './'  # Chemin où extraire le fichier .csv

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [14]:
import pandas as pd

In [16]:
pd.read_csv("./data/Competitions.csv").head()

,Id,Slug,Title,Subtitle,HostSegmentTitle,ForumId,OrganizationId,EnabledDate,DeadlineDate,ProhibitNewEntrantsDeadlineDate,...,CanQualifyTiers,TotalTeams,TotalCompetitors,TotalSubmissions,ValidationSetName,ValidationSetValue,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,HostName,CompetitionTypeId
0,2408,Eurovision2010,Forecast Eurovision Voting,This competition requires contestants to forec...,Featured,2,NaN,04/07/2010 07:57:43,05/25/2010 18:00:00,NaN,...,False,22,25,22,NaN,NaN,False,False,NaN,1
1,2435,hivprogression,Predict HIV Progression,This contest requires competitors to predict t...,Featured,1,NaN,04/27/2010 21:29:09,08/02/2010 12:32:00,NaN,...,True,107,116,855,NaN,NaN,False,False,NaN,1
2,2438,worldcup2010,World Cup 2010 - Take on the Quants,Quants at Goldman Sachs and JP Morgan have mod...,Featured,3094129,NaN,06/03/2010 08:08:08,06/11/2010 13:29:00,NaN,...,False,0,0,0,NaN,NaN,False,False,NaN,1
3,2439,informs2010,INFORMS Data Mining Contest 2010,The goal of this contest is to predict short t...,Featured,4,NaN,06/21/2010 21:53:25,10/10/2010 02:28:00,NaN,...,True,145,153,1483,NaN,NaN,False,False,NaN,1
4,2442,worldcupconf,World Cup 2010 - Confidence Challenge,The Confidence Challenge requires competitors ...,Featured,3,NaN,06/03/2010 08:08:08,06/11/2010 13:28:00,NaN,...,False,63,64,63,NaN,NaN,False,False,NaN,1


In [17]:

comp_slug = "Eurovision2010"
comp_url = "https://www.kaggle.com/c/"+str(comp_slug)
comp_url

'https://www.kaggle.com/c/Eurovision2010'

# Scrapping

In [22]:
from langchain.document_loaders import AsyncHtmlLoader
urls = [comp_url]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.34it/s]


In [24]:
docs[0]

Document(page_content='\r\n\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n\r\n<head>\r\n  <title>Forecast Eurovision Voting  | Kaggle</title>\r\n  <meta charset="utf-8" />\r\n    <meta name="robots" content="index, follow" />\r\n  <meta name="description" content="This competition requires contestants to forecast the voting for this year&#x27;s Eurovision Song Contest in Norway on May 25th, 27th and 29th. " />\r\n  <meta name="turbolinks-cache-control" content="no-cache" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">\r\n  <meta name="theme-color" content="#008ABC" />\r\n  <script nonce="53d9Z7YCloSoXWbCeMIDJQ==" type="text/javascript">\r\n    window["pageRequestStartTime"] = 1695309355526;\r\n    window["pageRequestEndTime"] = 1695309355561;\r\n    window["initialPageLoadStartTime"] = new Date().getTime();\r\n  </script>\r\n  <link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous" /><link

In [37]:
from langchain.document_loaders import AsyncChromiumLoader
import nest_asyncio
nest_asyncio.apply() # for use in notebook

loader = AsyncChromiumLoader(["http://www.kaggle.com/c/Eurovision2010"])
html = loader.load()
html

[Document(page_content='Error: Protocol error (Page.navigate): Cannot navigate to invalid URL\n=========================== logs ===========================\nnavigating to "www.kaggle.com/c/Eurovision2010", waiting until "load"\n============================================================', metadata={'source': 'www.kaggle.com/c/Eurovision2010'})]

In [34]:
html

[Document(page_content='Error: net::ERR_CERT_AUTHORITY_INVALID at https://www.kaggle.com/c/Eurovision2010\n=========================== logs ===========================\nnavigating to "https://www.kaggle.com/c/Eurovision2010", waiting until "load"\n============================================================', metadata={'source': 'https://www.kaggle.com/c/Eurovision2010'})]

In [51]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(ignore_https_errors=True)
        page = await context.new_page()
        await page.goto('http://www.kaggle.com/c/Eurovision2010')
        results = await page.content()
        await browser.close()
        print(results)

asyncio.run(main())

<!DOCTYPE html><html lang="en"><head>
  <title>Forecast Eurovision Voting  | Kaggle</title>
  <meta charset="utf-8">
    <meta name="robots" content="index, follow">
  <meta name="description" content="This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th. ">
  <meta name="turbolinks-cache-control" content="no-cache">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">
  <meta name="theme-color" content="#008ABC">
  <script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.IoxrLNdlTyI.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo9N48n3oloz8UTxoCozKcpUKaADkg/cb=gapi.loaded_0?le=scs" nonce="" async=""></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1695312806729;
    window["pageRequestEndTime"] = 1695312807320;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
  <link

In [67]:
from selenium import webdriver
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options) 
driver.get('http://www.kaggle.com/c/Eurovision2010')

driver.implicitly_wait(10)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [68]:
soup

<html lang="en"><head>
<title>Forecast Eurovision Voting  | Kaggle</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th. " name="description"/>
<meta content="no-cache" name="turbolinks-cache-control"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color"/>
<script async="" nonce="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.L92w_vMR5kE.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-GjmyR8TQVDKPrbqj5UQg7cU3cCA/cb=gapi.loaded_0?le=scs"></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1695366581616;
    window["pageRequestEndTime"] = 1695366581849;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<link crossorigin="anonymous" href="

In [69]:
driver.page_source

'<html lang="en"><head>\n  <title>Forecast Eurovision Voting  | Kaggle</title>\n  <meta charset="utf-8">\n    <meta name="robots" content="index, follow">\n  <meta name="description" content="This competition requires contestants to forecast the voting for this year\'s Eurovision Song Contest in Norway on May 25th, 27th and 29th. ">\n  <meta name="turbolinks-cache-control" content="no-cache">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">\n  <meta name="theme-color" content="#008ABC">\n  <script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.L92w_vMR5kE.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-GjmyR8TQVDKPrbqj5UQg7cU3cCA/cb=gapi.loaded_0?le=scs" nonce="" async=""></script><script nonce="" type="text/javascript">\n    window["pageRequestStartTime"] = 1695366581616;\n    window["pageRequestEndTime"] = 1695366581849;\n    window["initialPageLoadStartTime"] = new Date().getTime();\n  </script>\n  <link